In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
from __future__ import print_function, division
import copy
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torchvision.transforms import ToTensor,Resize,Normalize,RandomHorizontalFlip,RandomVerticalFlip,CenterCrop
from sklearn.metrics import accuracy_score,precision_recall_fscore_support,roc_curve,auc,roc_auc_score,classification_report
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True
plt.ion()   # interactive mode

In [3]:
print(torch.cuda.device_count())
print(torch.cuda.current_device())

1
0


Run for the first iteration to generate base Global Model

In [4]:

method = 'FedAvgIVL'
i = 10
learningRate = 0.001
c1  = 1/0.298793  ##BRACS1
c2 = 1/0.302705 ##BRACS2
c3 = 1/0.134646 ##BreakHis1
c4 = 1/0.097750  ##BreakHis2
#dd '''2525 246 1439'''
base_weight_dir = '/DATA1/Mangaldeep/V3/Weight/FedAvgIVL'
base_log_dir = '/DATA1/Mangaldeep/V3/Log/FedAvgIVL'
avg_weight_dir = base_weight_dir+'/'+'avg'+str(i)+'.pth'
logfilepath = base_log_dir+'/'+str(method)+'_'+str(i)+'.txt'
if i >0 :
    previous_avg_dir = base_weight_dir+'/'+'avg'+str(i-1)+'.pth'

In [5]:
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

if i == 0:
    model_ft = models.efficientnet_b3(weights=None)
    num_ftrs = model_ft.classifier[1].in_features
    model_ft.classifier[1] = nn.Linear(num_ftrs, 2)
    torch.save(model_ft.state_dict(),avg_weight_dir)

cuda:0


In [6]:
data1_dir = '/DATA1/Mangaldeep/V3/Dataset/BRACS1/test'
data2_dir = '/DATA1/Mangaldeep/V3/Dataset/BRACS2/test'
data3_dir = '/DATA1/Mangaldeep/V3/Dataset/BreakHis1/test'
data4_dir = '/DATA1/Mangaldeep/V3/Dataset/BreakHis2/test'

In [7]:
weight1_dir = '/DATA1/Mangaldeep/V3/Weight/FedAvgIVL/FedAvgIVL_BRACS1_'+str(i-1)+'_1323.pth'
weight2_dir = '/DATA1/Mangaldeep/V3/Weight/FedAvgIVL/FedAvgIVL_BRACS2_'+str(i-1)+'_1323.pth'
weight3_dir = '/DATA1/Mangaldeep/V3/Weight/FedAvgIVL/FedAvgIVL_BreakHis1_'+str(i-1)+'_697.pth'
weight4_dir = '/DATA1/Mangaldeep/V3/Weight/FedAvgIVL/FedAvgIVL_BreakHis2_'+str(i-1)+'_698.pth'

In [8]:
weight1 = torch.load(weight1_dir,map_location=torch.device('cpu'))
weight2 = torch.load(weight2_dir,map_location=torch.device('cpu'))
weight3 = torch.load(weight3_dir,map_location=torch.device('cpu'))
weight4 = torch.load(weight4_dir,map_location=torch.device('cpu'))
weight_avg = torch.load(weight3_dir,map_location=torch.device('cpu'))


total = c1 + c2+ c3 +c4
c1x = c1/total
c2x = c2/total
c3x = c3/total
c4x = c4/total
print(c1x+c2x+c3x+c4x)



for key in weight1:
  weight_avg[key] = c1x*weight1[key] + c3x*weight3[key] + c2x*weight2[key] + c4x*weight4[key]

1.0


In [9]:
torch.save(weight_avg,avg_weight_dir)

In [10]:
data_transforms = {
    'train': transforms.Compose([
        Resize((256,256),interpolation=transforms.InterpolationMode.BICUBIC),
        transforms.RandomCrop(224),
        #transforms.RandomResizedCrop(size = (224,224)),
        #transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(p=0.3),
        transforms.RandomVerticalFlip(p=0.3),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        #transforms.Resize(256),
        Resize((256,256),interpolation=transforms.InterpolationMode.BICUBIC),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

model_ft = models.efficientnet_b3(weights=None)
num_ftrs = model_ft.classifier[1].in_features
model_ft.classifier[1] = nn.Linear(num_ftrs, 2)

model_ft.load_state_dict(torch.load(avg_weight_dir))
#model_ft = model_ft.to(device)


model_ft = model_ft.to(device)
model_ft.eval()
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
#optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.0001, momentum=0.9)
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.0002,weight_decay=0.00001)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

# BreakHis1

In [11]:
test_dataset = datasets.ImageFolder(data3_dir, transform=data_transforms['val'])
class_names = test_dataset.classes
test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=1,
        num_workers=0,
        shuffle=False
    )
model_ft.eval()
with torch.no_grad():
  y_true=[]
  y_pred=[]
  y_probas = []
  for i, (inputs, labels) in enumerate(test_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    y_true.append(labels.detach().cpu().numpy())
    model_ft.to(device)
    outputs = model_ft(inputs)
    outputs = nn.Softmax(dim=1)(outputs)
    max, preds = torch.max(outputs, 1)
    y_probas.append(outputs.cpu().numpy())
    y_pred.append(int(preds.detach().cpu().numpy()))


y_true = np.array(y_true)
y_pred = np.array(y_pred)
prob = np.array(y_probas)
acc = accuracy_score(y_true,y_pred)
y_probas = prob.reshape(prob.shape[0],prob.shape[2])
precision,recall,fscore,_=precision_recall_fscore_support(y_true, y_pred, average='macro')
auc_score = roc_auc_score(y_true, y_probas[:,1])
precision,recall,fscore,_=precision_recall_fscore_support(y_true, y_pred, average='macro')
print(f"BreakHis1 accuracy = {round(acc,4)} , precision = {round(precision,4)},recall = {round(recall,4)},fscore = {round(fscore,4)},,auc_score = {round(auc_score,4)}")
print(classification_report(y_true,y_pred))
with open (logfilepath, 'a') as file:
    file.writelines(f"Learning rate {learningRate}")
    file.writelines('\n')
    file.writelines(data3_dir)
    file.writelines('\n')
    file.writelines(f"BreakHis1 accuracy = {round(acc,4)} , precision = {round(precision,4)},recall = {round(recall,4)},fscore = {round(fscore,4)},auc_score = {round(auc_score,4)}")
    file.writelines('\n')
    file.writelines(classification_report(y_true,y_pred))
    file.writelines('\n')

BreakHis1 accuracy = 0.865 , precision = 0.9066,recall = 0.79,fscore = 0.8212,,auc_score = 0.9877
              precision    recall  f1-score   support

           0       0.97      0.59      0.73        63
           1       0.84      0.99      0.91       137

    accuracy                           0.86       200
   macro avg       0.91      0.79      0.82       200
weighted avg       0.88      0.86      0.85       200



# BreakHis2

In [12]:
test_dataset = datasets.ImageFolder(data4_dir, transform=data_transforms['val'])
class_names = test_dataset.classes
test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=1,
        num_workers=0,
        shuffle=False
    )
model_ft.eval()
with torch.no_grad():
  y_true=[]
  y_pred=[]
  y_probas = []
  for i, (inputs, labels) in enumerate(test_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    y_true.append(labels.detach().cpu().numpy())
    model_ft.to(device)
    outputs = model_ft(inputs)
    outputs = nn.Softmax(dim=1)(outputs)
    max, preds = torch.max(outputs, 1)
    y_probas.append(outputs.cpu().numpy())
    y_pred.append(int(preds.detach().cpu().numpy()))


y_true = np.array(y_true)
y_pred = np.array(y_pred)
prob = np.array(y_probas)
acc = accuracy_score(y_true,y_pred)
y_probas = prob.reshape(prob.shape[0],prob.shape[2])
precision,recall,fscore,_=precision_recall_fscore_support(y_true, y_pred, average='macro')
auc_score = roc_auc_score(y_true, y_probas[:,1])
precision,recall,fscore,_=precision_recall_fscore_support(y_true, y_pred, average='macro')
print(f"BreakHis2 accuracy = {round(acc,4)} , precision = {round(precision,4)},recall = {round(recall,4)},fscore = {round(fscore,4)},,auc_score = {round(auc_score,4)}")
print(classification_report(y_true,y_pred))
with open (logfilepath, 'a') as file:
    file.writelines(f"Learning rate {learningRate}")
    file.writelines('\n')
    file.writelines(data3_dir)
    file.writelines('\n')
    file.writelines(f"BreakHis2 accuracy = {round(acc,4)} , precision = {round(precision,4)},recall = {round(recall,4)},fscore = {round(fscore,4)},auc_score = {round(auc_score,4)}")
    file.writelines('\n')
    file.writelines(classification_report(y_true,y_pred))
    file.writelines('\n')


BreakHis2 accuracy = 0.8507 , precision = 0.9107,recall = 0.7619,fscore = 0.7947,,auc_score = 0.9777
              precision    recall  f1-score   support

           0       1.00      0.52      0.69        63
           1       0.82      1.00      0.90       138

    accuracy                           0.85       201
   macro avg       0.91      0.76      0.79       201
weighted avg       0.88      0.85      0.83       201



# BRACS1

In [13]:
test_dataset = datasets.ImageFolder(data1_dir, transform=data_transforms['val'])
class_names = test_dataset.classes
test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=1,
        num_workers=0,
        shuffle=False
    )
model_ft.eval()
with torch.no_grad():
  y_true=[]
  y_pred=[]
  y_probas = []
  for i, (inputs, labels) in enumerate(test_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    y_true.append(labels.detach().cpu().numpy())
    model_ft.to(device)
    outputs = model_ft(inputs)
    outputs = nn.Softmax(dim=1)(outputs)
    max, preds = torch.max(outputs, 1)
    y_probas.append(outputs.cpu().numpy())
    y_pred.append(int(preds.detach().cpu().numpy()))


y_true = np.array(y_true)
y_pred = np.array(y_pred)
prob = np.array(y_probas)
acc = accuracy_score(y_true,y_pred)
y_probas = prob.reshape(prob.shape[0],prob.shape[2])
precision,recall,fscore,_=precision_recall_fscore_support(y_true, y_pred, average='macro')
auc_score = roc_auc_score(y_true, y_probas[:,1])
precision,recall,fscore,_=precision_recall_fscore_support(y_true, y_pred, average='macro')
print(f"BRACS1 accuracy = {round(acc,4)} , precision = {round(precision,4)},recall = {round(recall,4)},fscore = {round(fscore,4)},,auc_score = {round(auc_score,4)}")
print(classification_report(y_true,y_pred))
with open (logfilepath, 'a') as file:
    file.writelines(data1_dir)
    file.writelines('\n')
    file.writelines(f"BRACS1 accuracy = {round(acc,4)} , precision = {round(precision,4)},recall = {round(recall,4)},fscore = {round(fscore,4)},,auc_score = {round(auc_score,4)}")
    file.writelines('\n')
    file.writelines(classification_report(y_true,y_pred))
    file.writelines('\n')


BRACS1 accuracy = 0.5049 , precision = 0.667,recall = 0.577,fscore = 0.463,,auc_score = 0.7444
              precision    recall  f1-score   support

           0       0.88      0.19      0.31       121
           1       0.45      0.96      0.61        83

    accuracy                           0.50       204
   macro avg       0.67      0.58      0.46       204
weighted avg       0.71      0.50      0.44       204



# BRACS2

In [14]:
test_dataset = datasets.ImageFolder(data2_dir, transform=data_transforms['val'])
class_names = test_dataset.classes
test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=1,
        num_workers=0,
        shuffle=False
    )
model_ft.eval()
with torch.no_grad():
  y_true=[]
  y_pred=[]
  y_probas = []
  for i, (inputs, labels) in enumerate(test_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    y_true.append(labels.detach().cpu().numpy())
    model_ft.to(device)
    outputs = model_ft(inputs)
    outputs = nn.Softmax(dim=1)(outputs)
    max, preds = torch.max(outputs, 1)
    y_probas.append(outputs.cpu().numpy())
    y_pred.append(int(preds.detach().cpu().numpy()))


y_true = np.array(y_true)
y_pred = np.array(y_pred)
prob = np.array(y_probas)
acc = accuracy_score(y_true,y_pred)
y_probas = prob.reshape(prob.shape[0],prob.shape[2])
precision,recall,fscore,_=precision_recall_fscore_support(y_true, y_pred, average='macro')
auc_score = roc_auc_score(y_true, y_probas[:,1])
precision,recall,fscore,_=precision_recall_fscore_support(y_true, y_pred, average='macro')
print(f"BRACS2 accuracy = {round(acc,4)} , precision = {round(precision,4)},recall = {round(recall,4)},fscore = {round(fscore,4)},,auc_score = {round(auc_score,4)}")
print(classification_report(y_true,y_pred))
with open (logfilepath, 'a') as file:
    file.writelines(data1_dir)
    file.writelines('\n')
    file.writelines(f"BRACS2 accuracy = {round(acc,4)} , precision = {round(precision,4)},recall = {round(recall,4)},fscore = {round(fscore,4)},,auc_score = {round(auc_score,4)}")
    file.writelines('\n')
    file.writelines(classification_report(y_true,y_pred))
    file.writelines('\n')


BRACS2 accuracy = 0.4902 , precision = 0.6953,recall = 0.5684,fscore = 0.4345,,auc_score = 0.7151
              precision    recall  f1-score   support

           0       0.95      0.15      0.26       121
           1       0.44      0.99      0.61        83

    accuracy                           0.49       204
   macro avg       0.70      0.57      0.43       204
weighted avg       0.74      0.49      0.40       204

